# Unsupervised learning of a single parameter

Before running this notebook: ensure that the steps outlined in sv_setup_instructions.txt are complete.

In [ ]:
import pydpf
import numpy as np
import torch
import pathlib
import sv_model
from tqdm import tqdm
from time import time
from sv_training_loop import train
import pandas as pd
import os

## Experiment Settings

In [ ]:
n_repeats = 10
training_epochs = 10
batch_size_test = 128
batch_size_train = 32
#Setting the true parameters here will only affect the testing of the ability to learn alpha and not the standard deviation of the gradients which is locked with alpha = 0.91.
true_alpha = 0.91
true_beta = 0.5
true_sigma = 1.
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
result_path = pathlib.Path('.').parent.absolute().joinpath('results/single_parameter_results.csv')
data_path = pathlib.Path('.').parent.absolute().joinpath('data/')
temp_data_path =  data_path / "temp.csv"
experiments = ['DPF', 'Soft', 'Stop-Gradient', 'Marginal Stop-Gradient', 'Optimal Transport', 'Kernel']
if not (data_path / "test_trajectory.csv").exists():
    raise FileNotFoundError("No file test_trajectory.csv found in the data directory, please download it from out github before running this notebook.")

In [ ]:
def get_DPF(DPF_type, SSM, rng):
    if DPF_type == 'DPF':
        return pydpf.DPF(SSM=SSM, resampling_generator=rng)
    if DPF_type == 'Soft':
        return pydpf.SoftDPF(SSM=SSM, resampling_generator=rng)
    if DPF_type == 'Stop-Gradient':
        return pydpf.StopGradientDPF(SSM=SSM, resampling_generator=rng)
    if DPF_type == 'Marginal Stop-Gradient':
        return pydpf.MarginalStopGradientDPF(SSM=SSM, resampling_generator=rng)
    if DPF_type == 'Optimal Transport':
        return pydpf.OptimalTransportDPF(SSM=SSM, regularisation=0.5, transport_gradient_clip=1.)
    if DPF_type == 'Kernel':
        kernel = pydpf.KernelMixture(pydpf.MultivariateGaussian(torch.zeros(1, device=device),torch.nn.Parameter(torch.eye(1, device=device)*0.1), generator=rng), generator=rng)
        return pydpf.KernelDPF(SSM=SSM, kernel=kernel)
    raise ValueError('DPF_type should be one of the allowed options')

## Timing and testing the variance of the gradients on a single given trajectory

In [ ]:
def test_gradients(experiment):
    experiment_cuda_rng = torch.Generator(device).manual_seed(0)
    aggregation_function_dict = {'ELBO': pydpf.LogLikelihoodFactors()}
    test_dataset = pydpf.StateSpaceDataset(data_path=data_path / "test_trajectory.csv", state_prefix='state', device='cuda')
    gradients = []
    size = 0
    alpha_p = torch.nn.Parameter(torch.tensor([[0.93]], dtype = torch.float32, device=device))
    SSM = sv_model.make_SSM(torch.tensor([[1.]], device=device), alpha_p, torch.tensor([0.5], device=device), device)
    DPF = get_DPF(experiment, SSM, experiment_cuda_rng)
    forward_time = []
    backward_time = []
    state = test_dataset.state[:,0:1].expand((101, batch_size_test, 1)).contiguous()
    observation = test_dataset.observation[:,0:1].expand((101, batch_size_test, 1)).contiguous()
    for i in tqdm(range(2560//batch_size_test)):
        DPF.update()
        size += state.size(1)
        torch.cuda.synchronize()
        start = time()
        outputs = DPF(observation=observation, n_particles=100, ground_truth=state, aggregation_function=aggregation_function_dict, time_extent=100)
        ls = torch.mean(outputs['ELBO'], dim=0)
        loss = ls.mean()
        torch.cuda.synchronize()
        forward_time.append((time() - start))
        alpha_p.grad = None
        for i in range(len(ls)):
            ls[i].backward(retain_graph=True)
            gradients.append(alpha_p.grad.item())
            alpha_p.grad = None
        #Free the stored tensors
        torch.cuda.synchronize()
        start = time()
        loss.backward()
        torch.cuda.synchronize()
        backward_time.append((time() - start))
    return forward_time, backward_time, gradients

In [ ]:
def test_learning_alpha(experiment):
    alphas = np.empty(n_repeats)
    for n in range(n_repeats):
        experiment_cuda_rng = torch.Generator(device).manual_seed(n*10)
        experiment_cpu_rng = torch.Generator().manual_seed(n*10)
        generation_rng = torch.Generator(device).manual_seed(n*10)
        true_SSM = sv_model.make_SSM(torch.tensor([[true_sigma]], device=device), torch.tensor([[true_alpha]], device=device), torch.tensor([true_beta], device=device), device, generation_rng)
        pydpf.simulate_and_save(temp_data_path, SSM=true_SSM, time_extent=1000, n_trajectories=500, batch_size=100, device=device, bypass_ask=True)
        alpha = torch.nn.Parameter(torch.rand((1,1), device=device, generator=experiment_cuda_rng), requires_grad=True)
        SSM = sv_model.make_SSM(torch.tensor([[1.]], device=device), alpha, torch.tensor([0.5], device=device), device, generation_rng)
        dpf = get_DPF(experiment, SSM, experiment_cuda_rng)
        if experiment == 'Kernel':
            opt = torch.optim.SGD([{'params':[alpha], 'lr':0.05}, {'params':dpf.resampler.mixture.parameters(), 'lr':0.01}])
        else:
            opt = torch.optim.SGD([{'params':[alpha], 'lr':0.05}])
        opt_schedule = torch.optim.lr_scheduler.ExponentialLR(opt, 0.95)
        dataset = pydpf.StateSpaceDataset(temp_data_path, state_prefix='state', device=device)
        _, ELBO = train(dpf, opt, dataset, training_epochs, (100, 100, 100), (batch_size_train, batch_size_test, batch_size_test), (0.5, 0.25, 0.25), 1., experiment_cpu_rng, target='ELBO', time_extent=100, lr_scheduler=opt_schedule)
        alphas[n] = alpha
        os.remove(temp_data_path)
    return alphas


In [ ]:
for experiment in experiments:
    print(f"Testing {experiment}")
    results = pd.read_csv(result_path, index_col=0)
    ft, bt, grads = test_gradients(experiment)
    alpha_list = test_learning_alpha(experiment)
    #Ignore first and last times because the last batch will have a different size to the rest, and cuda is often slower on the first iteration
    row = np.array([sum(ft[1:-1])/(len(ft)-2), sum(bt[1:-1])/(len(bt)-2), np.sqrt(np.var(grads)), np.mean(np.abs(alpha_list - 0.91))])
    results.loc[experiment] = row
    print(results)
    results.to_csv(result_path)